In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import math

class HandDetector:
    def __init__(self, mode=False, max_hands=2, detection_confidence=0.5, tracking_confidence=0.5):
        self.mode = mode
        self.max_hands = max_hands
        self.detection_confidence = detection_confidence
        self.tracking_confidence = tracking_confidence

        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=self.mode,
                                         max_num_hands=self.max_hands,
                                         min_detection_confidence=self.detection_confidence,
                                         min_tracking_confidence=self.tracking_confidence)
        self.mp_draw = mp.solutions.drawing_utils

    def find_hands(self, img, draw=True):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(img_rgb)

        if self.results.multi_hand_landmarks and draw:
            for hand_landmarks in self.results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(img, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)

    def find_position(self, img, hand_number=0, draw=True):
        lm_list = []

        if self.results.multi_hand_landmarks:
            selected_hand = self.results.multi_hand_landmarks[hand_number]

            for id, landmark in enumerate(selected_hand.landmark):
                h, w, _ = img.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                lm_list.append([id, cx, cy])

                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        return lm_list

def main():
    cap = cv2.VideoCapture(0)
    detector = HandDetector()

    while True:
        success, img = cap.read()

        detector.find_hands(img)
        lm_list = detector.find_position(img)

        if len(lm_list) != 0:
            # Use the position of the tip of the index finger, the base of the thumb, and the tip of the middle finger
            x_tip, y_tip = lm_list[8][1:]
            x_base, y_base = lm_list[4][1:]
            x_middle_tip, y_middle_tip = lm_list[12][1:]

            # Calculate the distance between the tip of the index finger and the base of the thumb
            distance = math.sqrt((x_tip - x_base)**2 + (y_tip - y_base)**2)

            # Check if the thumb and index finger are close for a click
            if distance < 50:
                pyautogui.click()
            else:
                # Check if the middle finger is below the index finger for scrolling
                if y_middle_tip > y_tip:
                    pyautogui.scroll(1)  
                else:
                    pyautogui.scroll(-1)  

            pyautogui.moveTo(x_tip, y_tip)

        cv2.imshow("AI Virtual Mouse", img)
        if cv2.waitKey(1) & 0xFF == 27:  
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
